In [1]:
from scipy.optimize import minimize
import numpy as np
import pandas as pd
from qiskit.circuit.library import efficient_su2
from Quantum_Games import game_circuit, player, prisoners_dilemma_payoff_calculator, get_aer_result

In [2]:
params = [(i*4*np.pi, i*4*np.pi) for i in np.linspace(0, 1, 10)]
strategies = [efficient_su2(1, reps=0).assign_parameters(parameters=param) for param in params]
n_players = 2

In [4]:
from qiskit.circuit.library import UnitaryGate

long = UnitaryGate([[1, 0], [0, 1]])
short = UnitaryGate([[0, 1], [1, 0]])
quantum_long = UnitaryGate([[1j, 0], [0, -1j]])
quantum_short = UnitaryGate([[0, -1j],[-1j, 0]])
strategies = [long, quantum_long, quantum_short, short]


In [5]:
def payoff(theta):
    players = []

    probs = theta.tolist()
    norm = sum(probs)

    if norm >= 1:
        return norm
    
    probs.append(1-norm)
    print(probs)
    for i in range(n_players):
        players.append(player(name = f'player_{i}', strategies=strategies, probabilities=probs))

    #num_strats = len(strategies)
    #probs_assignments = [theta[i:i + num_strats] for i in range(0, len(theta), num_strats)]
    #for i in range(len(probs_assignments)):
        #players.append(player(name = f'player_{i}', strategies=strategies, probabilities=probs_assignments[i]))
    
    circuit = game_circuit(players=players)
    result = get_aer_result(circuit)
    result_df = pd.DataFrame(result)
    payoffs = prisoners_dilemma_payoff_calculator(result_df)
    ave_payoff = -payoffs.values.mean()
    print(ave_payoff)
    return ave_payoff


In [6]:
x0 = np.random.rand(2*n_players)
x0 = x0/sum(x0)
x0 = x0 [:-1]
print(x0)
constraints = {'type': 'ineq', 'fun': lambda x: 1 - np.sum(x)}
bounds = [(1e-6, 1.0) for _ in range(len(x0))] 
result = minimize(fun=payoff,
                  method='SLSQP',
                  bounds=bounds,
                  constraints=constraints,
                  x0=x0)

[0.2964439  0.15479967 0.14305925]
[0.2964438968125692, 0.15479967099321654, 0.14305924773908993, 0.40569718445512426]
-0.5
[0.2964439117137304, 0.15479967099321654, 0.14305924773908993, 0.40569716955396307]
-1.0
[0.2964438968125692, 0.15479968589437773, 0.14305924773908993, 0.40569716955396307]
-0.0
[0.2964438968125692, 0.15479967099321654, 0.14305926264025112, 0.40569716955396307]
-0.0
[0.6482214643278366, 0.07740044200510146, 0.07153022230009548, 0.20284787136696658]
-0.5
[0.47233268057020283, 0.116100056499159, 0.1072947350195927, 0.3042725279110454]
-0.5
[0.384388288691386, 0.13544986374618778, 0.12517699137934132, 0.35498485618308484]
-1.0
[0.34041609798648326, 0.14512476621798945, 0.13411811849485245, 0.3803410173006748]
-1.0
[0.3184300026340324, 0.14996221745389013, 0.1385886820526079, 0.39301909785946954]
-0.5
[0.3074369497233008, 0.15238094422355333, 0.14082396489584892, 0.39935814115729695]
-0.5
[0.301940423267935, 0.15359030760838494, 0.1419416063174694, 0.40252766280621066

In [7]:
result

 message: Inequality constraints incompatible
 success: False
  status: 4
     fun: -0.0
       x: [ 2.971e-01  1.546e-01  1.429e-01]
     nit: 2
     jac: [-3.355e+07 -3.355e+07 -3.355e+07]
    nfev: 18
    njev: 2

In [8]:
def generate_initial_simplex(theta_0, perturbation_size=0.05):
    """
    Generates an initial simplex for the Nelder-Mead algorithm.

    Parameters:
        theta_0 (np.array): Initial guess (starting point) as a 1D array.
        perturbation_size (float): Size of the perturbation for each dimension.

    Returns:
        np.array: A (n+1, n) array representing the initial simplex.
    """
    n = len(theta_0)  # Number of dimensions
    simplex = np.zeros((n + 1, n))  # Initialize simplex matrix

    # First vertex is the initial guess
    simplex[0] = theta_0

    # Generate the remaining vertices by perturbing each dimension
    for i in range(n):
        vertex = theta_0.copy()
        vertex[i] += perturbation_size  # Perturb the i-th dimension
        simplex[i + 1] = vertex

    return simplex

In [11]:
x0 = np.random.rand(2*n_players)
x0 = x0/sum(x0)
x0 = x0 [:-1]
initial_simplex = generate_initial_simplex(x0, perturbation_size=0.5)
result = minimize(fun=payoff,
                  x0 = x0,
                  method='Nelder-Mead',
                  bounds = bounds,
                  tol=1e-2,
                  options = {'maxiter': 100,
                             'initial_simplex' : initial_simplex,
                            }
                 )

[0.16280900543526727, 0.22668651505834908, 0.2147234743718683, 0.3957810051345153]
-0.0
[0.7183645609908229, 1e-06, 0.07157515812395607, 0.21005928088522097]
-0.0
[0.25540159802786, 0.29778506057500526, 1e-06, 0.44681234139713466]
-0.5
[0.05169789432415639, 0.3333343333333333, 1e-06, 0.6149667723425103]
-0.0
[0.2615744375340327, 1e-06, 0.1908654216638829, 0.5475591408020845]
-0.0
[0.32021641284267466, 1e-06, 0.1431493162479122, 0.5366332709094132]
-0.5
[1e-06, 0.3496473837555696, 0.16700736895589763, 0.48334424728853276]
-1.0
[1e-06, 0.5244705756333544, 0.2147234743718684, 0.2608049499947771]
-0.0
[0.2209370018117558, 0.20493578116203415, 1e-06, 0.57412621702621]
-0.0
[0.1773410045293894, 0.22124883158427033, 0.15905468471990247, 0.4423554791664378]
-1.0
[1e-06, 0.5791198506098968, 0.07422605286928785, 0.34665309652081533]
-1.0
[1e-06, 0.4688923167248192, 0.26685773769672533, 0.2642489455784556]
-0.0
[0.1572579664354949, 0.34056187461245874, 0.06671518442418133, 0.43546497452786503]
-1

In [12]:
result

       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: -1.0
             x: [ 1.000e-06  3.496e-01  1.670e-01]
           nit: 16
          nfev: 44
 final_simplex: (array([[ 1.000e-06,  3.496e-01,  1.670e-01],
                       [ 4.915e-03,  3.494e-01,  1.639e-01],
                       [ 1.000e-06,  3.568e-01,  1.641e-01],
                       [ 5.543e-03,  3.456e-01,  1.668e-01]]), array([-1.000e+00, -1.000e+00, -1.000e+00, -1.000e+00]))